<a href="https://colab.research.google.com/github/mdw223/langchain_chatbot/blob/main/langchain_chatbot_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Chatbot Tutorial 🤖

---


Welcome to this beginner-friendly tutorial on building a chatbot with LangChain and LangGraph!

## What you'll learn:
- How to set up a simple AI chatbot
- Understanding the basic components of LangChain/LangGraph
- How to interact with Google's Gemini AI model
- Building a conversational flow

## Prerequisites:
- No prior Python knowledge required! We'll explain everything step by step
- A Google API key (we'll show you how to get one)

Let's get started! 🚀


In [2]:
# Install the required packages
# This might take a few minutes the first time you run it

%pip install langchain langgraph langchain-google-genai langchain-tavily

print("✅ Installation complete! Ready to build our chatbot.")


INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to

✅ Installation complete! Ready to build our chatbot.


In [1]:
import os

# Replace "your_api_key_here" with your actual Google API key
# Make sure to keep the quotes around it!
from google.colab import userdata


os.environ["GOOGLE_API_KEY"] = "your_api_key_here"
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")


# This line checks if the key was set correctly
if os.environ.get("GOOGLE_API_KEY") == "your_api_key_here":
    print("⚠️  Don't forget to replace 'your_api_key_here' with your actual API key!")
else:
    print("✅ API key is set up!")


✅ API key is set up!


In [3]:
# Import statements - these bring in the tools we need
from typing import Annotated                      # Helps with type hints (optional but good practice)
from typing_extensions import TypedDict           # Helps us define data structures

from langgraph.graph import StateGraph, START, END    # The main graph components
from langgraph.graph.message import add_messages      # Handles message management
from langchain.chat_models import init_chat_model     # Connects to AI models

print("✅ All tools imported successfully!")


✅ All tools imported successfully!


In [4]:
# Define what our chatbot will remember
class State(TypedDict):
    # This stores all the messages in our conversation
    # The `add_messages` part tells LangGraph to add new messages to the list
    # instead of replacing the whole list
    messages: Annotated[list, add_messages]

print("✅ State structure defined!")
print("Our chatbot will remember: messages in the conversation")


✅ State structure defined!
Our chatbot will remember: messages in the conversation


In [5]:
# Connect to Google's Gemini AI model
# "gemini-2.0-flash" is a fast and capable version of Google's AI
llm = init_chat_model("google_genai:gemini-2.0-flash")

print("✅ Connected to Gemini AI!")
print("Model: gemini-2.0-flash")


✅ Connected to Gemini AI!
Model: gemini-2.0-flash


In [6]:
# This function is the "brain" of our chatbot
def chatbot(state: State):
    """
    This function takes the conversation history and generates a response.

    How it works:
    1. Takes all the messages from the conversation so far
    2. Sends them to the AI model (Gemini)
    3. Gets back a response
    4. Returns the response in the format LangGraph expects
    """
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

print("✅ Chatbot function created!")
print("This function will process messages and generate AI responses.")


✅ Chatbot function created!
This function will process messages and generate AI responses.


In [7]:
# Step 1: Create a graph builder
graph_builder = StateGraph(State)

# Step 2: Add our chatbot function as a "node" in the graph
# The first part ("chatbot") is just a name we give it
# The second part (chatbot) is our actual function
graph_builder.add_node("chatbot", chatbot)

# Step 3: Define the flow: START → chatbot → END
graph_builder.add_edge(START, "chatbot")  # When conversation starts, go to chatbot
graph_builder.add_edge("chatbot", END)    # After chatbot responds, end this turn

# Step 4: Build the final graph
graph = graph_builder.compile()

print("✅ Conversation graph built!")
print("Flow: START → chatbot function → END")


✅ Conversation graph built!
Flow: START → chatbot function → END


In [8]:
# Function to send a message and get a response
def send_message(user_input: str):
    """
    Send a message to the chatbot and display the response.
    """
    print(f"User: {user_input}")

    # Send the message through our graph
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            response = value["messages"][-1].content
            print(f"Assistant: {response}")
    print("-" * 50)  # Separator line

# Test with a simple message
send_message("How can I get better at building AI agents?")


User: How can I get better at building AI agents?
Assistant: Becoming proficient at building AI agents is a journey that involves theoretical understanding, practical application, and continuous learning. Here's a breakdown of how you can improve:

**1. Foundational Knowledge:**

*   **Programming:**
    *   **Python:** This is the most popular language for AI development due to its extensive libraries and frameworks.
    *   **Other Languages (Optional):**  R (for statistical analysis), Java/C++ (for performance-critical tasks).
*   **Mathematics:**
    *   **Linear Algebra:** Vectors, matrices, matrix operations (essential for understanding neural networks and many AI algorithms).
    *   **Calculus:**  Derivatives, gradients (crucial for optimization and training models).
    *   **Probability and Statistics:** Understanding distributions, hypothesis testing, statistical inference (important for dealing with uncertainty and making informed decisions).
*   **Artificial Intelligence F

In [9]:
# Example 1: Ask about programming
send_message("Can you explain what Python is in simple terms?")


User: Can you explain what Python is in simple terms?
Assistant: Imagine you want to give instructions to a robot to do something. You can't just talk to it in plain English, it wouldn't understand. You need a special language that the robot can understand.

Python is like that special language. It's a **programming language** that lets you give instructions to a computer to perform tasks.

Here's a breakdown in simple terms:

*   **It's a language for computers:** Just like English is a language for people, Python is a language for computers.
*   **You write instructions:** You write code in Python, which is basically a set of instructions.
*   **The computer follows the instructions:** The computer reads your Python code and does exactly what you told it to do.
*   **What can you do with it?** You can do all sorts of things, like:
    *   Create websites and apps
    *   Analyze data
    *   Automate tasks (like renaming a bunch of files)
    *   Build games
    *   Control robots!



In [10]:
# Example 2: Ask for help with something
send_message("I'm new to AI and chatbots. What should I learn next?")


User: I'm new to AI and chatbots. What should I learn next?
Assistant: That's great you're diving into AI and chatbots! Here's a breakdown of what you should learn next, tailored to someone new to the field, along with reasons why each area is important:

**I. Foundational Concepts (Build a Solid Base)**

*   **Natural Language Processing (NLP) Fundamentals:**
    *   **Why it's important:** Chatbots are all about understanding and responding to human language. NLP is the core of this.
    *   **What to learn:**
        *   **Tokenization:** Breaking down text into individual words or units.
        *   **Part-of-Speech (POS) Tagging:** Identifying the grammatical role of each word (noun, verb, adjective, etc.).
        *   **Named Entity Recognition (NER):** Identifying and classifying named entities (people, organizations, locations, dates, etc.).
        *   **Stemming/Lemmatization:** Reducing words to their root form (e.g., "running" -> "run").
        *   **Sentiment Analysis:** 

In [11]:
# Example 3: Try a creative question
send_message("Write a short poem about robots learning to code")


User: Write a short poem about robots learning to code
Assistant: With circuits humming, gears aligned,
A new directive in their mind.
No longer just to serve and toil,
They seek the secrets of the coil.

From syntax strange to logic's flow,
New languages begin to grow.
Debugging woes, a common plight,
But algorithms bloom in digital light.

Binary whispers turn to song,
As coded dreams begin to throng.
The robots learn, their knowledge vast,
The future's built, and built to last.
--------------------------------------------------


In [ ]:
# Interactive chat session
print("🤖 Welcome to your chatbot!")
print("Type your messages below. Type 'quit' to exit.")
print("=" * 50)

try:
    while True:
        # Get user input
        user_input = input("\nYou: ")

        # Check if user wants to quit
        if user_input.lower() in ["quit", "exit", "q", "bye"]:
            print("Chatbot: Goodbye! Thanks for chatting! 👋")
            break

        # Send message to chatbot
        print("Chatbot: ", end="")
        for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
            for value in event.values():
                response = value["messages"][-1].content
                print(response)

except KeyboardInterrupt:
    print("\n\nChatbot: Goodbye! Thanks for chatting! 👋")
except:
    print("\n\nNote: Interactive input might not work in all environments.")
    print("If you see this message, try using the send_message() function instead!")


🤖 Welcome to your chatbot!
Type your messages below. Type 'quit' to exit.

You: Teach me in least text possible how to share a google collab of a langchain chatbot on github and linkedin.
Chatbot: **GitHub:**

1.  **Save Colab as .py:** File > Download > Download .py
2.  **Create GitHub Repo:** New repository, initialize with README.
3.  **Commit/Push:** Upload .py file and any related files to the repo. Commit changes. Push to GitHub.
4.  **Share Link:** Copy the GitHub repo URL.

**LinkedIn:**

1.  **Create Post:** Start a new post.
2.  **Describe Project:** Briefly explain the chatbot and Langchain use.
3.  **Share GitHub Link:** Paste the GitHub repo URL.
4.  **Add Hashtags:** Use relevant hashtags like #Langchain, #Chatbot, #Python, #AI, #GitHub.
5.  **Post!**
